In [90]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType

In [2]:
spark = SparkSession.builder.getOrCreate()

In [4]:
dataset_path = "/datalake/raw"

In [9]:
df = spark.read.option("header", True).csv(f'{dataset_path}/dados_partidas_internacionais.csv')

In [136]:
partidas = df.withColumnRenamed("date", "data")\
                   .withColumnRenamed("home_team", "mandante")\
                   .withColumnRenamed("away_team", "visitante")\
                   .withColumnRenamed("home_score", "placar_mandante")\
                   .withColumnRenamed("away_score", "placar_visitante")\
                   .withColumnRenamed("tournament", "torneio")\
                   .withColumnRenamed("city", "cidade")\
                   .withColumnRenamed("country", "pais")\
                   .drop("neutral")\
                   .persist()

In [24]:
#Quantos registros existem na base?
total_registros = partidas.count()
print(f'TOTAL DE REGISTROS ENCONTRADOS NO DATAFRAME: {total_registros}')

TOTAL DE REGISTROS ENCONTRADOS NO DATAFRAME: 40839


In [26]:
#Quantas equipes únicas mandantes existem na base?
total_mandantes = partidas.select("mandante").distinct().count()
print(f'TOTAL DE REGISTROS ENCONTRADOS NO DATAFRAME: {total_mandantes}')

TOTAL DE REGISTROS ENCONTRADOS NO DATAFRAME: 309


In [42]:
#Quantas vezes as equipes mandantes saíram vitoriosas?
vitorias_mandante = partidas.filter(partidas.placar_mandante > partidas.placar_visitante)\
                            .count()
print(f'TOTAL DE VITORIAS DOS TIMES MANDANTES: {vitorias_mandante}')

TOTAL DE VITORIAS DOS TIMES MANDANTES: 19856


In [43]:
#Quantas vezes as equipes visitantes saíram vitoriosas?
vitorias_visitantes = partidas.filter(partidas.placar_visitante > partidas.placar_mandante)\
                              .count()
print(f'TOTAL DE VITORIAS DOS TIMES VISITANTES: {vitorias_visitantes}')

TOTAL DE VITORIAS DOS TIMES VISITANTES: 11552


In [39]:
#Quantas partidas resultaram em empate?
empates = partidas\
            .filter(partidas.placar_mandante == partidas.placar_visitante)\
            .count()
print(f'TOTAL DE PARTIDAS COM EMPATES: {empates}')

TOTAL DE PARTIDAS COM EMPATES: 9431


In [51]:
#Quantas partidas foram realizadas em cada país?
partidas.groupBy("pais").count().show()

+-------------------+-----+
|               pais|count|
+-------------------+-----+
|               Chad|   40|
|             Russia|  209|
|           Paraguay|  218|
|           Anguilla|   12|
|              Yemen|   58|
|U.S. Virgin Islands|   15|
|     Chinese Taipei|   47|
|            Senegal|  202|
|             Sweden|  637|
|             Guyana|   98|
|        Philippines|   83|
|              Burma|   79|
|             Jersey|   68|
|            Eritrea|   16|
| Netherlands Guyana|   29|
|           Djibouti|   21|
|              Tonga|   12|
|          Singapore|  368|
|           Malaysia|  644|
|               Fiji|  139|
+-------------------+-----+
only showing top 20 rows



In [112]:
#Qual país teve mais partidas
partidas_by_pais = partidas.groupBy("pais")\
                           .agg(F.count(F.col("data")).alias("total_partidas"))

print("EXIBINDO PAIS QUE MAIS TEVE PARTIDAS")
row = partidas_by_pais.sort(partidas_by_pais.total_partidas.desc()).first()
print(row)

EXIBINDO PAIS QUE MAIS TEVE PARTIDAS
Row(pais='United States', total_partidas=1144)


In [135]:
#Qual a partida com maior número de gols? / #Qual a maior goleada?
partidas = partidas.withColumn("quantidade_gols_partida", F.lit(F.col("placar_mandante") + F.col("placar_visitante"))\
                    .cast(IntegerType()))

partidas.sort(partidas.quantidade_gols_partida.desc()).first()

Row(data='2001-04-11', mandante='Australia', visitante='American Samoa', placar_mandante='31', placar_visitante='0', torneio='FIFA World Cup qualification', cidade='Coffs Harbour', pais='Australia', quantidade_gols_partida=31)

In [79]:
#Quantos jogos ocorreram no Brasil?
qtd_partidas_br = partidas.filter(partidas.pais == "Brazil").count()
print(f'TOTAL DE PARTIDAS NO BRASIL: {qtd_partidas_br}')

TOTAL DE PARTIDAS NO BRASIL: 529
